In [ ]:
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import time
from hazelcast import HazelcastClient

def connect_hazelcast():
    client = HazelcastClient(
        cluster_members=["192.168.1.22:5701"],
        cluster_name="dev",
        connection_timeout=5000,
        smart_routing=True,
        use_public_ip=True
    )
    print("Connected to the Hazelcast cluster.")
    return client

def generate_matrix(rows, cols):
    return np.random.randint(0, 100, size=(rows, cols))

def extract_block(matrix, start_row, start_col, size):
    return matrix[start_row:start_row + size, start_col:start_col + size]

def multiply_blocks(block_a, block_b):
    return np.dot(block_a, block_b)

def assemble_matrix(result_map, matrix_size, block_size):
    result_matrix = np.zeros((matrix_size, matrix_size), dtype=int)
    blocks = matrix_size // block_size

    for i in range(blocks):
        for j in range(blocks):
            block = np.array(result_map.get(f"{i}_{j}").result())
            for k in range(block_size):
                for l in range(block_size):
                    row = i * block_size + k
                    col = j * block_size + l
                    result_matrix[row][col] = block[k][l]

    return result_matrix

def distribute_blocks(client, matrix_a, matrix_b, block_size, sub_block_size):
    executor = client.get_executor("blockExecutor")
    result_map = client.get_map("resultMap")
    result_map.clear()

    matrix_size = matrix_a.shape[0]
    blocks = matrix_size // block_size

    with ThreadPoolExecutor() as pool:
        futures = []

        for i in range(blocks):
            for j in range(blocks):
                block_a = extract_block(matrix_a, i * block_size, j * block_size, block_size)
                block_b = extract_block(matrix_b, i * block_size, j * block_size, block_size)

                future = pool.submit(multiply_blocks, block_a, block_b)
                result_map.put(f"{i}_{j}", future.result())

    result_matrix = assemble_matrix(result_map, matrix_size, block_size)
    return result_matrix

def main():
    client = connect_hazelcast()
    
    if not client.lifecycle.is_running():
        print("Could not connect to the Hazelcast cluster. Check the configuration.")
        return

    matrix_size = 50
    block_size = 10
    sub_block_size = 5

    matrix_a = generate_matrix(matrix_size, matrix_size)
    matrix_b = generate_matrix(matrix_size, matrix_size)

    start_time = time.time()
    result = distribute_blocks(client, matrix_a, matrix_b, block_size, sub_block_size)
    end_time = time.time()

    print("Matrix multiplication completed!")
    print("Result (first 10 values):", result.flatten()[:10])
    print(f"Total time: {end_time - start_time:.2f} seconds")

    client.shutdown()

main()